# SageMaker Example

## 2. Build the container

demo codes are in `app/`
build and push the docker with following commands:

In [1]:
!bash build_and_push_sglang.sh

set -e

# This script shows how to build the Docker image and push it to ECR to be ready for use
# by SageMaker.

# The argument to this script is the region name. 
# 尝试使用 IMDSv2 获取 token
TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600")
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    56    0     0  45197      0 --:--:-- --:--:-- --:--:-- 56000

# Get the current region and write it to the backend .env file
region=$(curl -H "X-aws-ec2-metadata-token: $TOKEN" -s http://169.254.169.254/latest/meta-data/placement/region)
# region=$(aws configure get region)
suffix="com"

if [[ $region =~ ^cn ]]; then
    suffix="com.cn"
fi

# Get the account number associated with the current IAM credentials
account=$(aws sts  get-caller-identity --query Account --output text)

SGL_VERSION=latest
inference_image=sagema

## 3. Deploy on SageMaker

define the model and deploy on SageMaker


### 3.1 Init SageMaker session

In [1]:
# !pip install boto3 sagemaker transformers
import re
import json
import os,dotenv
import boto3
import sagemaker
from sagemaker import Model


dotenv.load_dotenv()
print(os.environ)

boto_sess = boto3.Session(
    region_name='us-east-1'
)

sess = sagemaker.session.Session(boto_session=boto_sess)
# role = sagemaker.get_execution_role()
role = os.environ.get('role')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ubuntu/.config/sagemaker/config.yaml
environ({'USER': 'ubuntu', 'SSH_CLIENT': '72.21.198.67 21575 22', 'XDG_SESSION_TYPE': 'tty', 'SHLVL': '2', 'HOME': '/home/ubuntu', 'OLDPWD': '/home/ubuntu/.vscode-server', 'SSL_CERT_FILE': '/usr/lib/ssl/cert.pem', 'DBUS_SESSION_BUS_ADDRESS': 'unix:path=/run/user/1000/bus', 'LOGNAME': 'ubuntu', '_': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin/python', 'XDG_SESSION_CLASS': 'user', 'XDG_SESSION_ID': '32521', 'VSCODE_CLI_REQUIRE_TOKEN': 'b6cfbca6-9b17-4767-9219-e97b4b919005', 'PATH': '/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/.vscode-server/cli/servers/Stable-e54c774e0add60467559eb0d1e229c6452cf8447/server/bin/remote-cli:/home/ubuntu/.local/bin:/home/ubuntu/workspace/llm_model_hub/miniconda3/envs/py311/bin:/home/ubuntu/workspace/llm_mod

### 3.2 Prepare model file

#### Option 2: deploy vllm by model_id

In [3]:
!tar czvf model.tar.gz model_tar/

model_tar/
model_tar/env
model_tar/s5cmd


In [2]:


s3_code_prefix = f"sagemaker_endpoint/sglang/"
bucket = sess.default_bucket() 
code_artifact = sess.upload_data("model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-434444145045/sagemaker_endpoint/sglang//model.tar.gz


### 3.3 Deploy model

In [3]:
# CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/vllm:v0.5.5'
# model = Model(
#     name=sagemaker.utils.name_from_base("sagemaker-vllm")+"_model",
#     model_data=code_artifact,
#     image_uri=CONTAINER,
#     role=role,
#     sagemaker_session=sess,
# )

# # 部署模型到endpoint
# endpoint_name = sagemaker.utils.name_from_base("sagemaker-vllm")+"_endpoint"
# print(f"endpoint_name: {endpoint_name}")
# predictor = model.deploy(
#     initial_instance_count=1,
#     instance_type='ml.g5.2xlarge',
#     endpoint_name=endpoint_name,
# )

### test deployment from s3

In [ ]:
CONTAINER='434444145045.dkr.ecr.us-east-1.amazonaws.com/sagemaker_endpoint/sglang:v0.4.3.post2-cu124'
model_path = "s3://sagemaker-us-east-1-434444145045/Qwen2-5-3B-Instruct/032650faedac452e86f95f3f3b004342/finetuned_model/"
model_id = 'Qwen/Qwen2-1.5B-Instruct'
env={
    # "HF_MODEL_ID": model_id,
    "S3_MODEL_PATH":model_path,
}
model = Model(
    name=sagemaker.utils.name_from_base("sagemaker-sglang")+"-model",
    model_data=code_artifact,
    image_uri=CONTAINER,
    role=role,
    sagemaker_session=sess,
    env=env,
)

# 部署模型到endpoint
endpoint_name = sagemaker.utils.name_from_base("sagemaker-sglang")+"-endpoint"
print(f"endpoint_name: {endpoint_name}")
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.g5.2xlarge',
    endpoint_name=endpoint_name,
)

endpoint_name: sagemaker-sglang-2025-02-19-01-35-37-565-endpoint


## 4. Test

you can invoke your model with SageMaker SDK

### 4.1 Message api non-stream mode

In [7]:
runtime = boto3.client('runtime.sagemaker',region_name='us-east-1')
endpoint_name = "sagemaker-sglang-2025-02-18-14-55-17-330-endpoint"
payload = {
    "messages": [
    {
        "role": "user",
        "content": "who are you"
    }
    ],
    "model":"qwen",
    "max_tokens": 1024,
    "stream": False
}
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["message"]["content"])

I am a large language model created by Alibaba Cloud. I am called Qwen.


### 4.2 Message api stream mode

In [8]:
payload = {
    "messages": [
    {
        "role": "user",
        "content": "Write a quick sort in python"
    }
    ],
    "model":"qwen",
    "max_tokens": 4096,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.span()[1]
            print(data["choices"][0]["delta"]["content"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]

Here is an implementation of the quick sort algorithm in Python:
```python
def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)
```

This function takes an array as input and returns a sorted version of it. It works by first checking if the array is already sorted, in which case it simply returns the array. Otherwise, it chooses a pivot element (here, the middle element of the array) and partitions the array into three parts: elements that are less than the pivot, elements that are equal to the pivot, and elements that are greater than the pivot. It then recursively sorts the left and right partitions and concatenates them with the middle partition to produce the final sorted array.

### 4.3 Completion api non-stream mode

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/deepseek-coder-6.7b-instruct", trust_remote_code=True)
messages=[
    { 'role': 'user', 'content': "write a quick sort algorithm in python."}
]
prompt = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

payload = {
    "model": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "prompt": prompt,
    "max_tokens": 1024,
    "stream": False
}

response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

print(json.loads(response['Body'].read())["choices"][0]["text"])

### 4.4 Completion api stream mode

In [ ]:
payload = {
    "model": "deepseek-ai/deepseek-coder-1.3b-instruct",
    "prompt": prompt,
    "max_tokens": 1024,
    "stream": True
}

response = runtime.invoke_endpoint_with_response_stream(
    EndpointName=endpoint_name,
    ContentType='application/json',
    Body=json.dumps(payload)
)

buffer = ""
for t in response['Body']:
    buffer += t["PayloadPart"]["Bytes"].decode()
    last_idx = 0
    for match in re.finditer(r'^data:\s*(.+?)(\n\n)', buffer):
        try:
            data = json.loads(match.group(1).strip())
            last_idx = match.end()
            # print(data)
            print(data["choices"][0]["text"], end="")
        except (json.JSONDecodeError, KeyError, IndexError) as e:
            pass
    buffer = buffer[last_idx:]
